In [16]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

In [3]:
data_df = pd.read_parquet("../dataset/train_df_all_english.parquet")

In [4]:
data_df.groupby("impact_length_idx").head(5).sort_values("impact_length_idx")

,url,Translation,impact_length_idx,language
4,https://www.esgtoday.com/eu-regulators-welcome...,"EU Regulators Welcome, Critique New European S...",0.0,English
16,https://www.esgtoday.com/eni-doubles-sustainab...,Eni Doubles Sustainability-Linked Bond Offerin...,0.0,English
19,https://www.esgtoday.com/bank-of-america-ceo-s...,Bank of America CEO: Sustainable Finance Deman...,0.0,English
21,https://www.esgtoday.com/republican-politician...,"Republican Politicians Target Proxy Firms ISS,...",0.0,English
22,https://www.esgtoday.com/guest-post-esg-isnt-a...,Guest Post: ESG isn’t About Altruism – it’s Ab...,0.0,English
0,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
1,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
2,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
7,https://www.esgtoday.com/red-states-sue-to-sto...,Red States Sue to Stop Biden Administration Ru...,1.0,English
11,https://www.esgtoday.com/methane-emissions-det...,Methane Emissions Detection Platform Kuva Rais...,1.0,English


In [13]:
data_augmentation_dict = {
    "less than 2 years": [data_df.loc[4]["Translation"], data_df.loc[19]["Translation"]],
    "between 2 and 5 years": [data_df.loc[7]["Translation"], data_df.loc[11]["Translation"]],
    "more than 5 years": [data_df.loc[6]["Translation"], data_df.loc[8]["Translation"]]
}

In [18]:
data_augmentation_messages = []

repeat_n = 2 # call api 2 more times

for impact_duration, text_list in data_augmentation_dict.items():

    template = f"""Give 10 examples of real news related to ESG (Environmental, Social, Governance) that has an impact duration {impact_duration}. Each example should have a news title and news summary, connected with "||". Generate these examples in english language. Two example news:

    "{text_list[0]}"
    
    "{text_list[1]}"
    
    The output should only contain a numbered list of news.
    
    """
    
    conversation = [
            {"role": "system", "content": "You are an ESG analyst, skilled assessing the level and duration an event in the news article might have on the company"},
            {"role": "user", "content": template}
        ]
    
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=conversation
    )
    
    message = completion.choices[0].message.content
    
    print(message)
    
    data_augmentation_messages.append(message)
    
    for i in range(repeat_n):
        assitant_message = {"role": "assistant", "content": message}
        new_user_message = {"role": "user", "content": "Give me 10 more real examples."}
        
        conversation.append(assitant_message)
        conversation.append(new_user_message)
        
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=conversation
        )
        
        
        
        
        
        

ChatCompletionMessage(content='1. "Tesla\'s Emissions Credits Revenue Faces Steep Declines || Tesla has been selling emissions credits to other companies unable to meet emissions regulations, however, with more companies investing in greener technologies, the demand for these credits is decreasing. Tesla may see a significant drop in this revenue stream in the coming year."\n\n2. "UK Insurance Companies Required to Divulge Climate Change Impact || The UK has made a legislation requiring insurance companies to clearly state their climate change impacts from activities and investments. This transparency may temporarily disrupt the insurance industry as new reporting systems are put in place."\n\n3. "Unilever Announces Plan to Disclose Suppliers\' Carbon Footprint || Unilever has made a commitment to disclose carbon emissions from its suppliers by the end of next year. This will cause short term disruptions as new transparency and monitoring systems are put in place."\n\n4. "Nike Faces La

In [34]:
augmented_dataframes = []

for idx, response in enumerate(data_augmentation_messages):
    augmented_news_list = response.content.split("\n\n")
    augmented_news_list = [i.split(". ", maxsplit=1)[1] for i in augmented_news_list]
    
    augmented_df = pd.DataFrame(augmented_news_list, columns=["sentence"])
    augmented_df["impact_length_idx"] = idx
    
    augmented_dataframes.append(augmented_df)
    
augmented_df = pd.concat(augmented_dataframes)

In [35]:
augmented_df.head()

,sentence,impact_length_idx
0,"""Tesla's Emissions Credits Revenue Faces Steep...",0
1,"""UK Insurance Companies Required to Divulge Cl...",0
2,"""Unilever Announces Plan to Disclose Suppliers...",0
3,"""Nike Faces Labor Unrest Due to Inadequate Wor...",0
4,"""Facebook Facing Data Privacy Litigation || Fa...",0


In [49]:
augmented_df["url"] = "ChatGPT augmented with Bing Search"
augmented_df["language"] = "English"
augmented_df[["title", "content"]] = augmented_df["sentence"].str.split(" \|\| ", expand=True)